To ensure the interactive widget works properly. Install ipywidgets and Node.Js

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
plt.rcParams.update({'text.usetex': True}) # import to make plots nicer
%config InlineBackend.figure_format = 'svg' # import to make plots nicer
from ODE_SOLVER import RungeKutta4 # for this import to work - make sure the file ODE_SOLVER is in the same file as your jupyter notebook

This program is based on the literature **"Sniffers, buzzers, toggles and blinkers: dynamics of regulatoryand signaling pathways in the cell" (Tyson 2002)** 

Implemented model 2b) Activator inhibitor <br>
$$
\begin{align}
\frac{dR}{dt} =& k_0 \cdot E_p(R) + k_1\cdot S - k_2 \cdot R - k^{\prime}_2\cdot X \cdot R \tag{1}\\
\frac{dX}{dt} =& k_5 \cdot R - k_6 \cdot X \tag{2}\\
E_p(R) =& G(k_3 \cdot R, k_4, J_3, J_4 \tag{3})\\
\end{align}
$$
Where G is defined as:
$$
\begin{align}
G(u, v, J, K) = \frac{2 \cdot u \cdot K}{v - u + v \cdot J + u \cdot K + \sqrt{(v-u+v\cdot J + u \cdot K)^2-4(v - u) \cdot u \cdot K}} \tag{4}
\end{align}
$$

**Parameter values** <br>
$k_0 = 4, k_1=k_2=k^{\prime}=k_3=k_4=1, k_5 = 0.1, k=6 = 0.075, J_3 = J_4 = 0.3, S=0.2$

In [2]:
def show_sol(k0=4., k1 = 1., k2 = 1., k2_p = 1., k3 = 1. , k4 = 1., k5=0.1, k6=0.075, J3 = 0.3, J4 = 0.3, S = 0.2):
    def G(u, v, J, K): # equation (4)
            numerator = 2 * u * K # numerator of the fraction
            den_1 = v - u + v * J + u * K # first part of the denominator
            den_2 = -4 * (v - u) * u * K # second part of the denominator
            radicand = den_1**2 + den_2 # expression under the square root
            denominator = den_1 + np.sqrt(radicand) # compute the entire denominator
            return numerator / denominator # compute and return the fraction

        R = x[0] 
        X = x[1]

        EP = G(k3*R, k4, J3, J4) # equation (3) 


        dRdT = k0*EP + k1*S - k2 * R - k2_p * X * R # equation (1)
        dXdT = (k5*R) - (k6*X) # equation (2)
        return [dRdT, dXdT]


    x0 = [0.1, 0.8] # initial conditions
    solver = RungeKutta4(CAMP_PDEP)
    solver.set_initial_condition(x0)
    time_points = np.linspace(0, 200, 1000)
    x, t = solver.solve(time_points)
    R = x[:,0]; X = x[:,1]
    plt.title("Some title")
    plt.plot(t, R, label='R')
    plt.plot(t, X, label='X')
    plt.xlabel('Time')
    plt.ylabel('Concenration')
    plt.legend()
w = interactive(show_sol, k0=(0.1, 10, 0.01), k1=(0.1, 10, 0.01),k2_p=(0.1, 10, 0.01),  k2=(0.1, 10, 0.01), k5=(0.01, 10, 0.01), k3=(0.1, 10, 0.01), k4=(0.1, 10, 0.01), k6=(0.01, 10, 0.01), J4=(0.1, 10, 0.01), J3=(0.1, 10, 0.01), S=(0.01, 1, 0.01),continuous_update=False)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 10)

In [3]:
w

interactive(children=(FloatSlider(value=4.0, description='k0', max=10.0, min=0.1, step=0.01), FloatSlider(valu…

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets



def CAMP_PDEP(t, x, k1, k3, k2, r1, r2, r3, r4, Km1, Km2, Km3, Km4, Dt, Et, ACt, PDEt):
    cAMP, PDEp = x
    def G(u, v, J, K): # equation (4)
            numerator = 2 * u * K # numerator of the fraction
            den_1 = v - u + v * J + u * K # first part of the denominator
            den_2 = -4 * (v - u) * u * K # second part of the denominator
            radicand = den_1**2 + den_2 # expression under the square root
            denominator = den_1 + np.sqrt(radicand) # compute the entire denominator
            return numerator / denominator # compute and return the fraction


    #ACp = (2 * r1 * cAMP * Km2) / (r2 * Dt * (1 + Km1 / ACt) + r1 * cAMP * (1 + Km2 / ACt) + np.sqrt((r2 * Dt * (1 + Km1 / ACt) + r1 * cAMP * (1 + Km2 / ACt)) ** 2 + 4 * (r1 * cAMP - r2 * Dt) ** 2))


    dPDEp  = r3 * cAMP * ((PDEt - PDEp) / Km3) - r4 * Et * PDEp / (Km4 + PDEp)

    dcAMP = (k1*(ACt*G(r1*cAMP, r2*Dt, Km1/ACt, Km2/ACt))) - (k3 + k2 * PDEp) * cAMP
    return np.array([dcAMP, dPDEp])

def plot_nullclines(k1, k3, k2, r1, r2, r3, r4, Km1, Km2, Km3, Km4, Dt, Et, ACt, PDEt):
    cAMP_vals = np.linspace(-30, 30, 100)
    dACp_vals = np.linspace(-30, 30, 100)
    cAMP_grid, dACp_grid = np.meshgrid(cAMP_vals, dACp_vals)

    t = 0
    x = np.array([cAMP_grid, dACp_grid])
    nullclines = CAMP_PDEP(t, x, k1, k3, k2, r1, r2, r3, r4, Km1, Km2, Km3, Km4, Dt, Et, ACt, PDEt)

    plt.figure(figsize=(8, 6))
    plt.contour(cAMP_grid,dACp_grid, nullclines[1], levels=[0], colors='r', linestyles='solid')
    plt.contour(cAMP_grid,dACp_grid ,nullclines[0], levels=[0], colors='b', linestyles='solid')
    plt.xlabel('cAMP')
    plt.ylabel('ACp')
    plt.show()


f = interactive(plot_nullclines,
                k1=widgets.FloatSlider(min=0, max=10, step=0.1, value=4.32),
                k3=widgets.FloatSlider(min=0, max=10, step=0.1, value=0.58),
                k2=widgets.FloatSlider(min=0, max=10, step=0.1, value=3.73),
                r1=widgets.FloatSlider(min=0, max=10, step=0.1, value=0.98),
                r2=widgets.FloatSlider(min=0, max=10, step=0.1, value=4.48),
                r3=widgets.FloatSlider(min=0, max=10, step=0.1, value=0.1),
                r4=widgets.FloatSlider(min=0, max=10, step=0.1, value=0.14),
                Km1=widgets.FloatSlider(min=0, max=10, step=0.1, value=2.48),
                Km2=widgets.FloatSlider(min=0, max=10, step=0.1, value=3.34),
                Km3=widgets.FloatSlider(min=0, max=10, step=0.1, value=9.24),
                Km4=widgets.FloatSlider(min=0, max=10, step=0.1, value=7.24),
                Dt=widgets.FloatSlider(min=0, max=10, step=0.1, value=2.12),
                Et=widgets.FloatSlider(min=0, max=10, step=0.1, value=1.6),
                ACt=widgets.FloatSlider(min=0, max=10, step=0.1, value=6.68),
                PDEt=widgets.FloatSlider(min=0, max=10, step=0.1, value=4.64),
                PDEp=widgets.FloatSlider(min=-30, max=30, step=0.1, value=1)
                )

f